# **Nutritional Data Analysis: Clean-up notebook**


## File import and intial observation of the dataset

In [168]:
# coding: utf-8
import pandas as pd
# read the csv file
df = pd.read_csv("foodfacts.csv", sep='\t',low_memory=False)

#Initial dataset observations

# Print the shape of df
print(df.shape)

# Print the columns of df
print(df.columns)







(320772, 162)
Index(['code', 'url', 'creator', 'created_t', 'created_datetime',
       'last_modified_t', 'last_modified_datetime', 'product_name',
       'generic_name', 'quantity',
       ...
       'ph_100g', 'fruits-vegetables-nuts_100g',
       'collagen-meat-protein-ratio_100g', 'cocoa_100g', 'chlorophyl_100g',
       'carbon-footprint_100g', 'nutrition-score-fr_100g',
       'nutrition-score-uk_100g', 'glycemic-index_100g',
       'water-hardness_100g'],
      dtype='object', length=162)


-  Remove Columns with insufficient data (more than 50% of products don't have the value) and columns that won't be retained for the analysis for the analysis
-  Remove all rows where the nutritional data is insufficient (NaN for all the necessary info) in the curated dataframe

In [169]:
missing_values_percent=(df.isnull().sum().sum()/(df.shape[0]*df.shape[1]))*100
print('{}%'.format(missing_values_percent))

76.22157263195135%


In [170]:
df_edited=df.copy()


for column in df.drop(['categories_fr','origins','manufacturing_places','main_category_fr'], axis=1).columns.values.tolist():
    
    total_rows=df_edited.shape[0]
    nonempty_rows=df_edited[[column]].count().sum()
    if (nonempty_rows/total_rows)<=0.5:
        del df_edited[column]

#remove incoherent data from these columns (values have to be positive and inferior to 100)
for column in ['fat_100g', 'saturated-fat_100g','carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g','salt_100g', 'sodium_100g']:
    df_edited.loc[df_edited[column]<0, column]=None
    df_edited.loc[df_edited[column]>100, column]=None
    


## Display the number of unique values for some of the columns we have retained:

This will give us an idea about the general distribution of the data in the dataset and the columns most relevant  to our analysis.

In [171]:
print(df_edited['brands'].value_counts())
print(df_edited['main_category_fr'].value_counts())
print(df_edited['brands_tags'].value_counts())
print(df_edited['countries'].value_counts())
print(df_edited['origins'].value_counts())
print(df_edited['ingredients_from_palm_oil_n'].value_counts())
print(df_edited['ingredients_that_may_be_from_palm_oil_n'].value_counts())

Carrefour                                      2978
Auchan                                         2340
U                                              2050
Meijer                                         1995
Leader Price                                   1700
Kroger                                         1660
Casino                                         1608
Ahold                                          1370
Spartan                                        1341
Roundy's                                       1299
Great Value                                    1230
Cora                                           1068
Weis                                           1053
Target Stores                                   947
Shoprite                                        944
Picard                                          906
Coop                                            842
Food Club                                       831
Harris Teeter                                   794
Monoprix    

0.0    237243
1.0     10037
2.0      1321
3.0       286
4.0        45
5.0         6
6.0         1
Name: ingredients_that_may_be_from_palm_oil_n, dtype: int64


### Removing the extraneous columns that we won't be using during the exploration

In [172]:
print(df_edited.columns)

for col in ['url', 'creator', 'created_t', 'created_datetime','last_modified_t', 'last_modified_datetime', 'serving_size', 'states', 'states_tags', 'states_fr','nutrition-score-uk_100g']:
    del df_edited[col]
df_edited=df_edited.dropna(thresh=3) 
print(df_edited.shape)
pd.DataFrame(df_edited.columns).style

Index(['code', 'url', 'creator', 'created_t', 'created_datetime',
       'last_modified_t', 'last_modified_datetime', 'product_name', 'brands',
       'brands_tags', 'categories_fr', 'origins', 'manufacturing_places',
       'countries', 'countries_tags', 'countries_fr', 'ingredients_text',
       'serving_size', 'additives_n', 'additives',
       'ingredients_from_palm_oil_n',
       'ingredients_that_may_be_from_palm_oil_n', 'nutrition_grade_fr',
       'states', 'states_tags', 'states_fr', 'main_category_fr', 'energy_100g',
       'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g',
       'fiber_100g', 'proteins_100g', 'salt_100g', 'sodium_100g',
       'nutrition-score-fr_100g', 'nutrition-score-uk_100g'],
      dtype='object')
(320685, 27)


,0
0,code
1,product_name
2,brands
3,brands_tags
4,categories_fr
5,origins
6,manufacturing_places
7,countries
8,countries_tags
9,countries_fr


### Look at the percentage of missing values for each column in the dataframe we now have

In [173]:
def get_percentage_missing(dframe):
    num = dframe.isnull().sum()
    den = len(dframe)
    return (round(num/den, 2)*100)

print(get_percentage_missing(df_edited).sort_values())
    

code                                        0.0
countries                                   0.0
countries_tags                              0.0
countries_fr                                0.0
product_name                                6.0
brands                                      9.0
brands_tags                                 9.0
proteins_100g                              19.0
energy_100g                                19.0
salt_100g                                  20.0
sodium_100g                                20.0
ingredients_that_may_be_from_palm_oil_n    22.0
ingredients_from_palm_oil_n                22.0
additives_n                                22.0
ingredients_text                           22.0
additives                                  22.0
fat_100g                                   24.0
carbohydrates_100g                         24.0
sugars_100g                                24.0
saturated-fat_100g                         28.0
nutrition_grade_fr                      

In [177]:
df_edited.drop_duplicates()
missing_values_percent_clean=(df_edited.isnull().sum().sum()/(df_edited.shape[0]*df_edited.shape[1]))*100
print('{}%'.format(missing_values_percent_clean))
print(df_edited.shape)

27.481196212505754%
(320685, 27)


### This is the dataframe we will keep for our exploratory analysis, we now export it to a new csv file

In [178]:
df_edited.to_csv('cleaned_foodfacts.csv',sep='\t',index=False,encoding='utf-8')